In [ ]:
# %pip install -q google-generativeai


In [ ]:
import google.generativeai as genai
import pandas as pd
import csv
import time

In [ ]:
genai.configure(api_key="APIKEY")
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)
    # """
    #         Kết quả   
    #         models/gemini-1.0-pro
    #         models/gemini-1.0-pro-001
    #         models/gemini-1.0-pro-latest
    #         models/gemini-1.0-pro-vision-latest
    #         models/gemini-pro
    #         models/gemini-pro-vision
    #   """


In [ ]:
# ----- cấu hình API -----
model = genai.GenerativeModel("gemini-2.5-flash")

In [ ]:
def extract_aspect(sentence, sentiment):
    prompt = f"""
Task: Extract exactly ONE aspect term from the sentence that corresponds to the given sentiment.
If no aspect clearly matches the sentiment, return exactly: None.

STRICT RULES:
- Output ONLY the aspect term (a noun or noun phrase).
- Do NOT output full sentences.
- Do NOT output multiple aspects.
- Do NOT explain.
- Do NOT infer sentiment; use the given sentiment only.
- The output must be ONLY one noun phrase or 'None'.

Sentence: "{sentence}"
Sentiment: {sentiment}
Aspect:
"""

    response = model.generate_content(prompt)
    return response.text.strip()


In [ ]:
output_path = r"H:\SentimentAnalystSchool\MidtermExam\DatasetUnder80kWord\AmazonsElectronics\output_with_aspects.csv"

with open(output_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["reviewText", "Sentiment", "Aspect"])


In [ ]:
df = pd.read_csv("H:\SentimentAnalystSchool\MidtermExam\DatasetUnder80kWord\AmazonsElectronics\AmazonsReviewDropColumns.csv")

for i, row in df.iterrows():
    sentence = row["reviewText"]
    sentiment = row["Sentiment"]

    # retry nếu ResourceExhausted
    for attempt in range(5):
        try:
            aspect = extract_aspect(sentence, sentiment)
            break
        except Exception as e:
            if "ResourceExhausted" in str(e):
                wait_time = 2 ** attempt  # exponential backoff: 1,2,4,8,16...
                print(f"{i}: Hit rate limit. Retry in {wait_time}s...")
                time.sleep(wait_time)
            else:
                aspect = "None"
                print(f"{i}: Error: {e}")
                break

    # Ghi ngay vào file CSV
    with open(output_path, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([sentence, sentiment, aspect])
    time.sleep(0.3)
    
    print(f"{i}: {aspect}")
print("Đã tạo file output_with_aspects.csv và ghi từng dòng thành công!")

